<a href="https://colab.research.google.com/github/Strix9289/How-to-use-library/blob/master/Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**クローリングとスクレイピング**

# urllib

In [ ]:
#ライブラリの読み込み
import urllib.request

## ダウンロードして直接保存

- **urlretrieve(url, savename)**：urlをダウンロードしてsavenameで現在のファイルに保存する

In [ ]:
#画像ファイルのアドレス
url = "https://www.library.osaka-u.ac.jp/img/main_image.png"
savename = "test.png"

urllib.request.urlretrieve(url, savename)
print("保存しました")

保存しました


## ダウンロードして一時的にメモリに保存
- **data = urlopen(url)**：
- data.read() : メモリのデータを読み込む
- data.decode('utf-8') : データを文字列に変換 



In [ ]:
#一時的にメモリ上に取得
mem = urllib.request.urlopen(url)

# 読み込む
mem = mem.read()

with open(savename, mode='wb') as f:
  f.write(mem)
  print('保存しました')

保存しました


## urljoin
**urljoin(base, "追加")**


# BeautigulSoup


In [6]:
from bs4 import BeautifulSoup

## HTMLの解析（構造が浅い場合）
- **soup = BeautifulSoup(html, 'html.parser')**  
- **h1 = soup.html.body.h1**
- **h1.string**

In [9]:
# ライブラリを取り込む
from bs4 import BeautifulSoup

In [14]:
# HTMLを解析する 
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# 解析したいHTML
html = """
<html><body>
  <h1>見出し</h1>
  <p>段落１</p>
  <p>段落2</p>
</body></html>
"""

In [16]:
# 任意の部分を抽出する 
h1 = soup.html.body.h1
p1 = soup.html.body.p
p2 = p1.next_sibling.next_sibling

# 要素のテキストを表示する
print("h1 = " + h1.string)
print("p  = " + p1.string)
print("p  = " + p2.string)

h1 = 見出し
p  = 段落１
p  = 段落2


## HTMLの解析（idで探す）
**find(id='title')**

In [19]:
html = """
<html><body>
  <h1 id="title">タイトル</h1>
  <p id="body">本文</p>
</body></html>
"""

# HTMLを解析する 
soup = BeautifulSoup(html, 'html.parser')

In [21]:
title = soup.find(id="title")
body  = soup.find(id="body")

# テキスト部分を表示 
print("title=" + title.string)
print("body="  + body.string)


title=タイトル
body=本文


## find_all

In [22]:
html = """
<html><body>
  <ul>
    <li><a href="http://first_url">first</a></li>
    <li><a href="http://second_url">second</a></li>
  </ul>
</body></html>
"""
# HTMLを解析する 
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# findAll()メソッドですべての<a>タグを取り出す
links = soup.find_all("a")

# リンク一覧を表示 (href属性はattrsプロパティにて、説明テキストはstringプロパティにて取得できる)
for a in links:
    href = a.attrs['href']
    text = a.string
    print(text, "->", href) 

first -> http://first_url
second -> http://second_url


## DOM要素の属性（タグ名の後にある各属性）を取り出す

In [26]:
html = "<p><a href='a.html'>test</a></p>"
soup = BeautifulSoup(html,"html.parser")

#変数aに<a>を代入
a = soup.p.a

#attrsプロパティの方はdict、#href属性の値を確認
type(a.attrs), a["href"]


(dict, 'a.html')

## CSSセレクタを使う
|メソッド|意味|
|:--|:--|
|soup.select_one()|CSSセレクタで要素を1つ取り出す|
|soup.select()|CSSセレクタで要素を複数取り出しリスト型にする|


In [31]:
# 解析対象となるHTML
html = """
<html><body>
<div id="mihon">
  <h1>タイトル</h1>
  <ul class="items">
    <li>段落1</li>
    <li>段落2</li>
    <li>段落3</li>
  </ul>
</div>
</body></html>
"""

# HTMLを解析する
soup = BeautifulSoup(html, 'html.parser')

In [32]:
# 必要な部分をCSSクエリで取り出す
# | タイトル部分を取得 --- (「>」で子要素にのみ適用)
h1 = soup.select_one("div#mihon > h1").string
print("h1 =", h1)

h1 = タイトル


In [35]:
# | リスト部分を取得 --- (「>」で子要素にのみ適用)
li_list = soup.select("div#mihon > ul.items > li")
for li in li_list:
	print("li =", li.string)

li = 段落1
li = 段落2
li = 段落3


## HTML内にあるリンクを抽出する


In [ ]:
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']") # CSSファイルを取得
    links += soup.select("a[href]") # リンク      # 参照先の相対パスをもつ<a>タグを取得
    result = []
    # href属性を取り出し、リンクを絶対パスに変換 --- <a>タグ内のhrefから絶対バスに変換
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

## ファイルをダウンロードし保存する

In [ ]:
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath): # ディレクトリならindex.html
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    # 既にダウンロード済み?
    if os.path.exists(savepath): return savepath
    # ダウンロード先のディレクトリを作成
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # ファイルをダウンロード --- 
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1) # 礼儀として1秒スリープ --- 
        return savepath
    except:
        print("ダウンロード失敗:", url)
        return None        

## 処理済み判断
HTMLではa->bへの参照とb->aの参照がある。

よって、一度参照したページを管理することで無限に同じページをダウンロードしないようにする。

In [ ]:
# 処理済みページを保存するリスト
proc_files = {}

In [ ]:
# HTMLを解析してダウンロードする関数 
def analize_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: return # 解析済みなら処理しない
    proc_files[savepath] = True
    print("analize_html=", url)
    # リンクを抽出 
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    for link_url in links:
        # リンクがルート以外のパスを指していたら無視 
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue #ただしCSSファイルの場合は読み込む
        # HTMLか？
        if re.search(r".(html|htm)$", link_url):
            # 再帰的にHTMLファイルを解析
            analize_html(link_url, root_url)
            continue
        # それ以外のファイル
        download_file(link_url)